# Getting the weather data of the airports

Using the table at http://www.flugzeuginfo.net/table_airportcodes_country-location_en.php
a csv with the IATA and ICAO codes of the airports can be created.

Using the ICAO codes the weather data can be retrieved from this website: https://www.wunderground.com/history/daily/de/frankfurt/EDDF/date/2015-3-18

Given a list with the destination airport codes and the schedules arrival time, a list of the corresponding weather conditions (wind speed, visibility) can be web scraped.

This list of weater conditions can then be incorporated into the dataframe with the delays and be subsequently used in a predictive model.

In [2]:
from datetime import datetime
import time
from os.path import exists

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [4]:
# The airport and date and time of departure
df_dep = pd.read_csv("data/sanitized_Train_data.csv", usecols=[3, 5], parse_dates=[1])
df_dep.columns = ['IATA', 'DATE_TIME']

In [5]:
df_dep.head()

,IATA,DATE_TIME
0,CMN,2016-01-03 10:30:00
1,MXP,2016-01-13 15:05:00
2,TUN,2016-01-16 04:10:00
3,DJE,2016-01-17 14:10:00
4,TUN,2016-01-17 14:30:00


In [6]:
# The airport and date and time of departure
df_dest = pd.read_csv("data/sanitized_Train_data.csv", usecols=[4, 6], parse_dates=[1])
df_dest.columns = ['IATA', 'DATE_TIME']

In [7]:
df_dest.head()

,IATA,DATE_TIME
0,TUN,2016-01-03 12:55:00
1,TUN,2016-01-13 16:55:00
2,IST,2016-01-16 06:45:00
3,NTE,2016-01-17 17:00:00
4,ALG,2016-01-17 15:50:00


In [8]:
df = pd.DataFrame()
df = pd.concat([df_dep, df_dest], axis=0)
df

,IATA,DATE_TIME
0,CMN,2016-01-03 10:30:00
1,MXP,2016-01-13 15:05:00
2,TUN,2016-01-16 04:10:00
3,DJE,2016-01-17 14:10:00
4,TUN,2016-01-17 14:30:00
...,...,...
107828,TUN,2018-07-06 02:00:00
107829,TUN,2018-01-13 09:00:00
107830,TUN,2018-11-07 12:50:00
107831,DJE,2018-01-23 18:45:00


In [9]:
df_airport_codes = pd.read_csv('data/list_IATA_ICAO_codes.csv', usecols=[0, 1])

In [10]:
code = df_airport_codes[df_airport_codes['IATA'] == 'CMN']['ICAO']
code = code.iloc[0]
type(code)

str

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215666 entries, 0 to 107832
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   IATA       215666 non-null  object        
 1   DATE_TIME  215666 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 4.9+ MB


In [12]:
df = df.join(df_airport_codes.set_index('IATA'), on='IATA', how='left', lsuffix='_left', rsuffix='_right')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215666 entries, 0 to 107832
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   IATA       215666 non-null  object        
 1   DATE_TIME  215666 non-null  datetime64[ns]
 2   ICAO       215666 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 6.6+ MB


In [14]:
df[df['ICAO'].isna()]# ['IATA'].unique()

,IATA,DATE_TIME,ICAO


In [15]:
df["ICAO"].unique()

array(['GMMN', 'LIMC', 'DTTA', 'DTTJ', 'LFBO', 'LTBA', 'LFPO', 'DTMB',
       'EBBR', 'DIAP', 'LIPZ', 'EHAM', 'EDDF', 'LEBL', 'OEJN', 'DAAG',
       'LPPT', 'LFST', 'LFLL', 'DFFD', 'EGKK', 'OLBA', 'LFMN', 'LPPR',
       'LFML', 'EDDL', 'DTTX', 'LIRF', 'LFPG', 'GQNN', 'LFRS', 'LSZH',
       'LSGG', 'GMFO', 'EDDM', 'EDDB', 'EDDH', 'GMMW', 'DTNH', 'HECA',
       'LYBE', 'LOWW', 'DAOO', 'LEMD', 'DTTZ', 'GABS', 'GOOY', 'HRYR',
       'LIPE', 'LMML', 'GMTA', 'EGLL', 'LFBD', 'LKPR', 'LJLJ', 'UUEE',
       'OEMA', 'LHBP', 'ESSA', 'EKCH', 'EBCI', 'EBOS', 'GMTT', 'DTTF',
       'LIRN', 'LKTB', 'LKMT', 'CYUL', 'DRRN', 'DAAT', 'HFFF', 'EDDK',
       'ORER', 'DTTG', 'DAAE', 'LTAI', 'GMMX', 'DXXX', 'LFQQ', 'LICJ',
       'FZQA', 'DTKA', 'LIRP', 'HSSS', 'LFLS', 'DABC', 'FOON', 'EYVI',
       'LTAC', 'FOOL', 'GUCY', 'ULLI', 'LZKZ', 'LZIB', 'OJAI', 'LROP',
       'LIMF', 'UKKK', 'HEBA', 'LIEE', 'UKBB', 'LGAV', 'LGTS', 'LTBJ',
       'GOBD', 'OTBD', 'DBBB', 'ELLX', 'FZAA', 'LPBJ', 'BIKF', 'DTTR',
      

In [16]:
df['DATE'] = df['DATE_TIME'].dt.date

In [17]:
#df.columns = ['IATA', 'DATE_TIME', 'ICAO', '', '', '', '', '']
# Adding columns for weather data
df['TIME_OBSERVATION'] = np.nan
df['TEMP'] = np.nan
df['WIND_SPEED'] = np.nan
df['PRECIP'] = np.nan
df['CONDITION'] = np.nan

In [18]:
df

,IATA,DATE_TIME,ICAO,DATE,TIME_OBSERVATION,TEMP,WIND_SPEED,PRECIP,CONDITION
0,CMN,2016-01-03 10:30:00,GMMN,2016-01-03,NaN,NaN,NaN,NaN,NaN
1,MXP,2016-01-13 15:05:00,LIMC,2016-01-13,NaN,NaN,NaN,NaN,NaN
2,TUN,2016-01-16 04:10:00,DTTA,2016-01-16,NaN,NaN,NaN,NaN,NaN
3,DJE,2016-01-17 14:10:00,DTTJ,2016-01-17,NaN,NaN,NaN,NaN,NaN
4,TUN,2016-01-17 14:30:00,DTTA,2016-01-17,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
107828,TUN,2018-07-06 02:00:00,DTTA,2018-07-06,NaN,NaN,NaN,NaN,NaN
107829,TUN,2018-01-13 09:00:00,DTTA,2018-01-13,NaN,NaN,NaN,NaN,NaN
107830,TUN,2018-11-07 12:50:00,DTTA,2018-11-07,NaN,NaN,NaN,NaN,NaN
107831,DJE,2018-01-23 18:45:00,DTTJ,2018-01-23,NaN,NaN,NaN,NaN,NaN


### Now the weather data have to be retrieved from the website using some web scraper

This will be done as follows:
1. Start with the first row in ```df```
1. Retrieve the weather data (unless they are already downloaded) and save them in weather_df
1. Find the weather observation closest to the given time
1. Add the weather observation to df
1. Continue with the next line in step 2. End when all lines have been processed.

In [4]:
path_weather_obs = "data/weather_obs.csv"

if exists(path_weather_obs):
    weather_obs = pd.read_csv(path_weather_obs, parse_dates=['DATE_TIME'])
else:
    weather_obs = pd.DataFrame(columns=['DATE_TIME', 'ICAO', 'TEMP', 'WIND_SPEED', 'PRECIP', 'CONDITION'])

In [ ]:
import time
from os.path import exists

from selenium import webdriver
from selenium.webdriver.common.by import By

path_weather_obs = "data/weather_obs.csv"

if exists(path_weather_obs):
    weather_obs = pd.read_csv(path_weather_obs, parse_dates=['DATE_TIME'])
else:
    weather_obs = pd.DataFrame(columns=['DATE_TIME', 'ICAO', 'TEMP', 'WIND_SPEED', 'PRECIP', 'CONDITION'])


count = 0

driver = webdriver.Chrome(executable_path='/home/fklein/zindi/Flight_Delay_Prediction_Challenge/chromedriver')  # Optional argument, if not specified will search path.


# search_box = driver.find_element_by_name('')
# <table _ngcontent-app-root-c202="" mat-table="" aria-labelledby="History observation" matsort="" aria-label="table of contents" class="mat-table cdk-table mat-sort ng-star-inserted" role="grid">
# <tr _ngcontent-app-root-c202="" role="row" mat-row="" class="mat-row cdk-row ng-star-inserted">

for index, row in df.iterrows():
    count += 1
    print(f"Count: {count}, Index: {index}")

    # if count > 2:
    #     break
    date_str = datetime.strftime(row['DATE_TIME'], '%Y-%m-%d')
    icao_code = row['ICAO']
    print(f"Date: {date_str}, ICAO: {icao_code}")

    # check if the data are already downloaded
    print("shape of the search result: ",weather_obs[(weather_obs['ICAO']==icao_code) & (pd.to_datetime(weather_obs['DATE_TIME']).dt.date==datetime.strptime(date_str, '%Y-%m-%d').date())].shape)
    if weather_obs[(weather_obs['ICAO']==icao_code) & (pd.to_datetime(weather_obs['DATE_TIME']).dt.date==datetime.strptime(date_str, '%Y-%m-%d').date())].shape[0] > 2:
        continue

    try:

        driver.get('https://www.wunderground.com/history/daily/de/frankfurt/'+icao_code+'/date/'+date_str);

        time.sleep(2) # Insert some pause to keep the traffic to the website low

        # weather_table = driver.find_elements_by_xpath(
        weather_table = driver.find_elements(by=By.XPATH, value=
                '//table[@class="mat-table cdk-table mat-sort ng-star-inserted"]/tbody/tr/td')

        table_html = list()
        table_html = list(weather_table)
        for ii in range(len(table_html)//10):
            dd = dict(
                zip(list(weather_obs.columns),[
                    [pd.to_datetime(datetime.strptime(date_str+'-'+table_html[0+ii*10].text, '%Y-%m-%d-%I:%M %p'))],
                    [icao_code],
                    [table_html[1+ii*10].text.split(' ')[0]],
                    [table_html[5+ii*10].text.split(' ')[0]],
                    [table_html[8+ii*10].text.split(' ')[0]],
                    [table_html[9+ii*10].text]
            ]))
            df_temp = pd.DataFrame(data=dd)
            weather_obs = pd.concat([weather_obs, df_temp])
        if len(table_html) == 0:
            dd = dict(
                zip(list(weather_obs.columns),[
                    [pd.to_datetime(datetime.strptime(date_str+'-'+'01:01 AM', '%Y-%m-%d-%I:%M %p'))],
                    [icao_code],
                    [999],
                    [999],
                    [999],
                    ['Not Applicable']
            ]))
            df_temp = pd.DataFrame(data=dd)
            for ii in range(0,3):
                weather_obs = pd.concat([weather_obs, df_temp])

        if count%20 == 0:
            weather_obs.to_csv(path_weather_obs, header=True, index=False)
            # print(f"Time: {table_html[0+ii*10].text}, Windspeed: {table_html[5+ii*10].text}, Conditions: {table_html[9+ii*10].text}")
    except Exception as e:
        print(e)
        break

driver.quit()

In [ ]:
weather_obs.to_csv(path_weather_obs, header=True, index=False)

In [8]:
already_processed = weather_obs[['DATE_TIME', 'ICAO']]
already_processed['DATE_TIME'] = already_processed['DATE_TIME'].apply(lambda x : x.date())
already_processed.drop_duplicates(inplace=True)
already_processed

/tmp/ipykernel_3085940/436968663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  already_processed['DATE_TIME'] = already_processed['DATE_TIME'].apply(lambda x : x.date())
/tmp/ipykernel_3085940/436968663.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  already_processed.drop_duplicates(inplace=True)


,DATE_TIME,ICAO
0,2016-01-13,LIMC
46,2016-01-16,DTTA
99,2016-01-17,DTTJ
145,2016-01-17,LFBO
193,2016-01-18,DTTA
...,...,...
56976,2016-08-23,GMMN
57028,2016-06-09,OEMA
57051,2016-06-09,OEJN
57075,2016-06-09,DTTJ
